<a href="https://colab.research.google.com/github/Ivan8Garcia/Proyecto_NoCountry/blob/main/validacionPKL_OXXN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# conectar a Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install onnx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 60.1 MB/s eta 0:00:00


In [ ]:
# Rutas Drive
path = '/content/drive/MyDrive/Colab Notebooks/Hackaton/Datos/flights_sample_3m.csv'

In [ ]:
import onnx

# Carga el modelo ONNX
model_path = "/content/drive/MyDrive/Colab Notebooks/Hackaton/Datos/flight_delay_model.onnx"
onnx_model = onnx.load(model_path)

# Imprime información básica del modelo
print("Versión del modelo ONNX:", onnx_model.ir_version)
print("Productor:", onnx_model.producer_name)
print("Versión del productor:", onnx_model.producer_version)
print("Dominio:", onnx_model.domain)
print("Versión del modelo:", onnx_model.model_version)

# Inspecciona los inputs (entradas)
print("\n--- Inputs ---")
for input_tensor in onnx_model.graph.input:
    print(f"Nombre: {input_tensor.name}")
    print(f"Tipo: {input_tensor.type}")
    # Para ver la forma (shape), si está disponible
    if input_tensor.type.tensor_type.shape:
        shape = [dim.dim_value if dim.dim_value else 'None' for dim in input_tensor.type.tensor_type.shape.dim]
        print(f"Forma: {shape}")
    print("---")

# Inspecciona los outputs (salidas)
print("\n--- Outputs ---")
for output_tensor in onnx_model.graph.output:
    print(f"Nombre: {output_tensor.name}")
    print(f"Tipo: {output_tensor.type}")
    if output_tensor.type.tensor_type.shape:
        shape = [dim.dim_value if dim.dim_value else 'None' for dim in output_tensor.type.tensor_type.shape.dim]
        print(f"Forma: {shape}")
    print("---")

# Inspecciona el grafo (nodos y operaciones)
print("\n--- Grafo (primeros 10 nodos) ---")
for i, node in enumerate(onnx_model.graph.node[:10]):  # Limita a 10 para no saturar
    print(f"Nodo {i}: {node.op_type} - Inputs: {node.input} - Outputs: {node.output}")

# Verifica si el modelo es válido
print("\n--- Verificación ---")
try:
    onnx.checker.check_model(onnx_model)
    print("El modelo ONNX es válido.")
except onnx.onnx_cpp2py_export.checker.ValidationError as e:
    print(f"Error de validación: {e}")

Versión del modelo ONNX: 10
Productor: skl2onnx
Versión del productor: 1.19.1
Dominio: ai.onnx
Versión del modelo: 0

--- Inputs ---
Nombre: DEP_HOUR
Tipo: tensor_type {
  elem_type: 1
  shape {
    dim {
    }
    dim {
      dim_value: 1
    }
  }
}

Forma: ['None', 1]
---
Nombre: DAY_OF_WEEK
Tipo: tensor_type {
  elem_type: 1
  shape {
    dim {
    }
    dim {
      dim_value: 1
    }
  }
}

Forma: ['None', 1]
---
Nombre: IS_WEEKEND
Tipo: tensor_type {
  elem_type: 1
  shape {
    dim {
    }
    dim {
      dim_value: 1
    }
  }
}

Forma: ['None', 1]
---
Nombre: DISTANCE
Tipo: tensor_type {
  elem_type: 1
  shape {
    dim {
    }
    dim {
      dim_value: 1
    }
  }
}

Forma: ['None', 1]
---
Nombre: TAXI_OUT
Tipo: tensor_type {
  elem_type: 1
  shape {
    dim {
    }
    dim {
      dim_value: 1
    }
  }
}

Forma: ['None', 1]
---
Nombre: AIRLINE
Tipo: tensor_type {
  elem_type: 8
  shape {
    dim {
    }
    dim {
      dim_value: 1
    }
  }
}

Forma: ['None', 1]
---

--- 

###Estructura Archivo Original PKL

In [ ]:
import joblib  # Usa joblib en lugar de pickle, ya que sklearn lo recomienda para modelos
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier

# Ruta del archivo .pkl (joblib puede cargar archivos .pkl)
path_org = "/content/drive/MyDrive/Colab Notebooks/Hackaton/Datos/flight_delay_model_backend.pkl"

# Carga el modelo desde el archivo .pkl usando joblib
try:
    model = joblib.load(path_org)
    print("Modelo cargado exitosamente desde el archivo .pkl.")

    # Imprime información básica del modelo (adaptado, ya que .pkl no tiene versión/productor como ONNX)
    print(f"Tipo de modelo: {type(model)}")
    if hasattr(model, '__version__'):
        print(f"Versión de sklearn (si aplica): {model.__version__}")
    else:
        print("Versión no disponible en el modelo.")

    # Extrae y imprime las columnas esperadas (basado en el preprocessor del Pipeline)
    if hasattr(model, 'named_steps_') and 'preprocessor' in model.named_steps_:
        preprocessor = model.named_steps_['preprocessor']
        if hasattr(preprocessor, 'transformers_'):
            # Asumiendo que el primer transformer es 'cat' y el segundo 'num' (como en tu output esperado)
            cat_features = preprocessor.transformers_[0][2]  # Lista de columnas categóricas
            num_features = preprocessor.transformers_[1][2]  # Lista de columnas numéricas
            expected_columns = cat_features + num_features
            print(f"Columnas esperadas: {expected_columns}")
        else:
            print("Columnas esperadas: No disponibles (preprocessor no tiene transformers_).")
    else:
        print("Columnas esperadas: No disponibles (no es un Pipeline con preprocessor).")

    # Imprime el modelo completo (Pipeline)
    print(f"Modelo (Pipeline): {model}")

    # Imprime los pasos del Pipeline
    print(f"Pasos del Pipeline: {model.steps}")

    # Inspecciona los inputs (entradas) - Basado en el Pipeline esperado
    print("\n--- Inputs ---")
    if hasattr(model, 'feature_names_in_'):
        print(f"Nombre: features")
        print(f"Tipo: tensor (asumido float64 o similar)")
        print(f"Forma: [None, {len(model.feature_names_in_)}]  # (n_samples, n_features)")
        print(f"Nombres de features: {list(model.feature_names_in_)}")
    elif hasattr(model, 'steps') and 'preprocessor' in dict(model.steps):
        # Extrae columnas del preprocessor si es un ColumnTransformer
        preprocessor = model.named_steps_['preprocessor']
        if hasattr(preprocessor, 'get_feature_names_out'):
            # Intenta obtener nombres de features transformadas
            try:
                feature_names = list(preprocessor.get_feature_names_out())
                print(f"Nombre: features")
                print(f"Tipo: tensor (asumido float64 o similar)")
                print(f"Forma: [None, {len(feature_names)}]  # (n_samples, n_features)")
                print(f"Nombres de features (transformadas): {feature_names}")
            except:
                print("Nombre: features")
                print("Tipo: tensor (asumido float64 o similar)")
                print("Forma: [None, n_features]  # n_features desconocido")
                print("Nota: No se pudieron obtener nombres de features transformadas.")
        else:
            print("Nombre: features")
            print("Tipo: tensor (asumido float64 o similar)")
            print("Forma: [None, n_features]  # n_features desconocido sin info adicional")
            print("Nota: Los nombres de features no están disponibles en este modelo .pkl.")
    else:
        print("Nombre: features")
        print("Tipo: tensor (asumido float64 o similar)")
        print("Forma: [None, n_features]  # n_features desconocido sin info adicional")
        print("Nota: Los nombres de features no están disponibles en este modelo .pkl.")
    print("---")

    # Inspecciona los outputs (salidas) - Basado en el clasificador
    print("\n--- Outputs ---")
    print(f"Nombre: predictions")
    print(f"Tipo: tensor (int o float, dependiendo del tipo de modelo)")
    if hasattr(model, 'classes_'):
        # Para modelos de clasificación
        print(f"Forma: [None, {len(model.classes_)}]  # (n_samples, n_classes) para clasificación")
        print(f"Clases: {list(model.classes_)}")
    else:
        # Para modelos de regresión o otros
        print("Forma: [None, 1]  # (n_samples, 1) para regresión u otros")
        print("Nota: No hay clases definidas (posiblemente un modelo de regresión).")
    print("---")

    # Inspecciona el "grafo" (nodos y operaciones) - Adaptado para modelos .pkl
    print("\n--- Grafo (tipo de modelo y pasos si es un pipeline) ---")
    print(f"Tipo de modelo: {type(model)}")
    if hasattr(model, 'steps'):
        # Si es un Pipeline de scikit-learn, inspecciona los pasos
        for i, (name, step) in enumerate(model.steps[:10]):  # Limita a 10 para no saturar
            print(f"Paso {i}: {name} - Tipo: {type(step)}")
            # Imprime detalles adicionales si es posible
            if hasattr(step, 'transformers_') and name == 'preprocessor':
                print(f"  Detalles del preprocessor: {step}")
            elif hasattr(step, 'estimators_') or hasattr(step, 'base_estimator_'):
                print(f"  Detalles del clasificador: {step}")
    else:
        print("No hay pasos de pipeline disponibles (modelo simple).")
        print("Nota: Los modelos .pkl no tienen un grafo explícito como ONNX; esto es una representación simplificada.")

    # Verifica si el modelo es válido (solo carga exitosa)
    print("\n--- Verificación ---")
    try:
        # Para scikit-learn, una verificación básica es intentar acceder a un atributo clave
        if hasattr(model, 'predict'):
            print("El modelo .pkl parece válido (tiene método predict).")
        else:
            print("Advertencia: El modelo no tiene método predict; podría no ser un modelo estándar.")
    except Exception as e:
        print(f"Error de verificación: {e}")

except Exception as e:
    print(f"Error al cargar o inspeccionar el modelo: {e}")

Modelo cargado exitosamente desde el archivo .pkl.
Tipo de modelo: <class 'dict'>
Versión no disponible en el modelo.
Columnas esperadas: No disponibles (no es un Pipeline con preprocessor).
Modelo (Pipeline): {'model': Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  ['AIRLINE']),
                                                 ('num', 'passthrough',
                                                  ['DEP_HOUR', 'DAY_OF_WEEK',
                                                   'IS_WEEKEND', 'DISTANCE',
                                                   'TAXI_OUT'])])),
                ('classifier',
                 RandomForestClassifier(class_weight='balanced', max_depth=15,
                                        n_jobs

###Análisis

### 1. Estructura General del Modelo

*   ONNX: Es una representación estandarizada del modelo (convertida
desde sklearn usando skl2onnx). Incluye metadatos como versión, productor (skl2onnx), y un grafo computacional que describe las operaciones paso a paso.
*   PKL: Es el modelo original de scikit-learn (sklearn), serializado como un objeto Python. Muestra la estructura del Pipeline directamente, con sus pasos y configuraciones.
*   Equivalencia: Ambos describen el mismo flujo: un preprocesamiento de datos seguido de un clasificador. El ONNX es esencialmente una "traducción" del PKL a un formato interoperable (ONNX), pero el núcleo (preprocesamiento + clasificación) es idéntico.

### 2. Inputs (Entradas)

*   ONNX: Lista 6 inputs explícitos con nombres (DEP_HOUR, DAY_OF_WEEK, IS_WEEKEND, DISTANCE, TAXI_OUT, AIRLINE). Cada uno tiene un tipo (tensor_type con elem_type 1 para numéricos/float o 8 para strings/categóricos) y forma ['None', 1] (indicando que acepta un lote de muestras, cada una con 1 valor por feature).
*   PKL: Muestra "Columnas esperadas" como una lista: ['DEP_HOUR', 'DAY_OF_WEEK', 'IS_WEEKEND', 'DISTANCE', 'TAXI_OUT', 'AIRLINE']. Esto coincide exactamente con los inputs del ONNX.
*   Equivalencia: Los inputs son los mismos 6 features. En el PKL, se derivan del ColumnTransformer en el preprocessor (que maneja categóricas y numéricas). El ONNX los representa de forma más detallada (con tipos y formas), pero son idénticos en contenido.

### 3. Outputs (Salidas)

*   ONNX: Dos outputs: output_label (etiqueta de clase, tipo tensor_type con elem_type 7 para int64, forma ['None']) y output_probability (probabilidades por clase, tipo sequence_type con map_type para un diccionario clave-valor).
*   PKL: No muestra outputs explícitos en el output proporcionado, pero el RandomForestClassifier (clasificador de bosque aleatorio) produce predicciones de clase y probabilidades por defecto en sklearn. El Pipeline implica que el output final es la predicción del clasificador.
*   Equivalencia: Ambos generan las mismas salidas lógicas (etiqueta + probabilidades). El ONNX las detalla más (con tipos ONNX), mientras que el PKL las asume como parte del comportamiento estándar del clasificador. Si ejecutas predicciones en el PKL, obtendrías predict() (etiquetas) y predict_proba() (probabilidades), que coinciden con el ONNX.

### 4. Grafo/Operaciones Internas (Procesamiento)

1.   ONNX: Describe un grafo con 8 nodos (operaciones secuenciales):
*   Nodos 0-4: Preprocesamiento (e.g., Concat para combinar numéricas, OneHotEncoder para AIRLINE, Reshape para ajustar formas).
*   Nodos 5-7: Clasificación (TreeEnsembleClassifier para el RandomForest, Cast y ZipMap para formatear outputs).
Esto refleja cómo skl2onnx convierte el Pipeline de sklearn a operaciones ONNX.

2.   PKL: Muestra el Pipeline con dos pasos principales:
*   preprocessor: Un ColumnTransformer que aplica OneHotEncoder a ['AIRLINE'] (categórica) y 'passthrough' a las numéricas (['DEP_HOUR', 'DAY_OF_WEEK', 'IS_WEEKEND', 'DISTANCE', 'TAXI_OUT']).
*   classifier: Un RandomForestClassifier con parámetros específicos (e.g., max_depth=15, class_weight='balanced').

3.   Equivalencia: El grafo del ONNX es una descomposición detallada del Pipeline del PKL. Por ejemplo:
*   El Concat y OneHotEncoder en ONNX corresponden al ColumnTransformer en PKL.
*   El TreeEnsembleClassifier en ONNX es el RandomForestClassifier en PKL.
*   Ambos manejan las mismas transformaciones: codificación one-hot para AIRLINE y paso directo para numéricas, seguido de clasificación.

### 5. Otras Consideraciones

1.   Metadatos: El ONNX incluye versiones y dominio (específicos de ONNX), mientras que el PKL muestra el tipo de objeto sklearn. Esto no afecta la equivalencia funcional.
2.   Validación: Ambos modelos son "válidos" en sus contextos (ONNX pasa onnx.checker, PKL se carga correctamente).
3.   Diferencias en Representación: El ONNX es más "bajo nivel" (grafo de operaciones) y portable (puedes ejecutarlo en otros frameworks), mientras que el PKL es el modelo sklearn nativo (más fácil de inspeccionar en Python). Pero funcionalmente, producen los mismos resultados para los mismos inputs.
4.   Prueba de Equivalencia: Si cargas datos de prueba y predices con ambos (usando onnxruntime para ONNX y joblib para PKL), las salidas deberían ser idénticas (o muy cercanas, considerando posibles diferencias numéricas menores en la conversión).